In [16]:
import random

In [253]:
N = 8
POPULATION_SIZE = 100
GENERATIONS = 10000
MUTATION_RATE = 0.8

In [262]:
# returns list of size N with all unique values
def generate_individual(N):
    individual = [x for x in range(N)]
    random.shuffle(individual)
    return individual

In [263]:
# test for generate_individual
for _ in range(5):
    print(generate_individual(N))

[5, 7, 3, 1, 4, 2, 6, 0]
[5, 2, 3, 7, 4, 1, 6, 0]
[6, 3, 7, 0, 5, 1, 2, 4]
[7, 4, 6, 1, 2, 0, 5, 3]
[1, 2, 7, 6, 3, 4, 5, 0]


In [264]:
# returns the fitness of a possible solution
# TODO: make this function work with arbitrary board sizes
def fitness(solution):
    attacking_queen_pairs = 0
    for i in range(N):
        for j in range(i+1, N):
            if solution[i] == solution[j] or abs(solution[i] - solution[j]) == abs(i - j):
                attacking_queen_pairs += 1
    #print(attacking_queen_pairs)
    return 28 - attacking_queen_pairs

In [265]:
# fitness function tests
a = [7,1,4,2,0,6,3,5]
print(fitness(a))
b = [1,2,3,4,5,6,7,8]
print(fitness(b))

28
0


In [279]:
# mutates possible solution
def mutate(solution):
    if random.random() > (1 - MUTATION_RATE):
        idx_1 = random.randrange(N)
        idx_2 = random.randrange(N)
        # swap the two random indices
        solution[idx_1], solution[idx_2] = solution[idx_2], solution[idx_1]
        return solution
    else:
        return solution

In [280]:
# mutation test
a = [1,2,3,4,5,6,7,8]
print(mutate(a))

[1, 2, 5, 4, 3, 6, 7, 8]


In [268]:
# one-point crossover function, returns two new children
def cut_and_crossover(solution1, solution2):
    cross_point = random.randrange(N)
    solution1[:cross_point], solution2[:cross_point] = solution2[:cross_point], solution1[:cross_point]
    return solution1, solution2

In [269]:
# tests of cut_and_crossover
a = [1,2,3,4,5,6,7,8]
b = [12,34,53,3,65,2,65,87]
print(cut_and_crossover(a,b))

([12, 34, 53, 3, 65, 6, 7, 8], [1, 2, 3, 4, 5, 2, 65, 87])


In [271]:
# finds and returns top two parents
def parent_selection(top_five):
    random.shuffle(top_five)
    top_five = sorted(top_five, key=fitness, reverse=True)
    #fitnesses = [fitness(x) for x in top_five]
    return top_five[:2]

In [272]:
# tests for parent_selection and with cut_and_crossover
a = [0, 7, 2, 4, 6, 1, 3, 5]
b = [4, 1, 3, 0, 7, 5, 2, 6]
c = [3, 0, 6, 1, 5, 7, 2, 4]
d = [2, 4, 6, 1, 3, 0, 7, 5]
e = [3, 6, 2, 7, 0, 4, 5, 1]

top = [a,b,c,d,e]
parents = parent_selection(top)
print(f"{parents=}")
parent1, parent2 = cut_and_crossover(parents[0], parents[1])
print(f"{parent1=}, {parent2=}")

parents=[[3, 0, 6, 1, 5, 7, 2, 4], [4, 1, 3, 0, 7, 5, 2, 6]]
parent1=[4, 1, 3, 0, 7, 5, 2, 4], parent2=[3, 0, 6, 1, 5, 7, 2, 6]


In [281]:
# the evolutionary algorithm, returns the best solution and what generation it was found in
def evo_alg(N):
    population = [generate_individual(N) for _ in range(POPULATION_SIZE)]
    
    for generation in range(GENERATIONS):
        # sort population by best fitness to worst
        population = sorted(population, key=fitness, reverse=True)
        
        # early termination if true solution found
        if fitness(population[0]) == 28:
            return population[0], generation
        
        # select top 5 parents, take best two and perform crossover for offspring
        parents = parent_selection(population[:5])
        child1, child2 = cut_and_crossover(parents[0], parents[1])
        
        # calculate fitness of children to then place into population
        child1_fitness = fitness(child1)
        child2_fitness = fitness(child2)
        
        
        for idx, individual in enumerate(population):
            if child1_fitness > fitness(individual):
                #print(child1_fitness, fitness(individual), idx, generation)
                population[idx] = child1
                break
                
        for idx, individual in enumerate(population):
            if child2_fitness > fitness(individual):
                population[idx] = child2
                break
    return population[0], GENERATIONS

In [286]:
# run the evolutionary algorithm
sol, gen = evo_alg(N)
print(sol, gen)
print(fitness(sol))

[2, 0, 6, 4, 1, 5, 7, 3] 10000
26


In [ ]:
successful_attempts = 0
generations_of_successful_attempts = []
for i in range(100):
    sol, gen = evo_alg(N)
    if fitness(sol) == 28:
        successful_attempts += 1
        generations_of_successful_attempts.append(gen)
        
print(f"Evolutionary Algorithm Success Rate: {successful_attempts/100}")
print(generations_of_successful_attempts)
    